<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkCatalystOptimiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('SparkCatalystOptimiser').getOrCreate()

In [2]:
spark

In [5]:
from pyspark.sql.types import *
data=[('Ram',2339943,'24-03-2022','04-03-1998',40000,'A','Marketing'),
      ('Raju',244335,'04-06-2021','22-08-1994',20000,'A','Sales'),
      ('Charan',254335,'29-03-2024','11-07-1996',25000,'A','Marketing'),
      ('Devdas',12343,'21-01-2021','04-05-1993',55000,'R','Sales'),
      ('Rezza',124335,'15-08-2024','03-12-1994',65000,'A','Sales')
      ]
schema=['Name','Emp_id','Joining_date','DOB','Salary','Status','Dept']
schem1=StructType([
    StructField('Name',StringType()),
    StructField('Emp_id',IntegerType()),
    StructField('Joining_date',DateType()),
    StructField('DOB',DateType()),
    StructField('Salary',IntegerType()),
    StructField('Status',StringType()),
    StructField('Dept',StringType())
])
df=spark.createDataFrame(data=data,schema=schema)
df.show()

+------+-------+------------+----------+------+------+---------+
|  Name| Emp_id|Joining_date|       DOB|Salary|Status|     Dept|
+------+-------+------------+----------+------+------+---------+
|   Ram|2339943|  24-03-2022|04-03-1998| 40000|     A|Marketing|
|  Raju| 244335|  04-06-2021|22-08-1994| 20000|     A|    Sales|
|Charan| 254335|  29-03-2024|11-07-1996| 25000|     A|Marketing|
|Devdas|  12343|  21-01-2021|04-05-1993| 55000|     R|    Sales|
| Rezza| 124335|  15-08-2024|03-12-1994| 65000|     A|    Sales|
+------+-------+------------+----------+------+------+---------+



In [6]:
result = df.filter(df.Status=='A').groupBy(df.Dept).count()
result.show()

+---------+-----+
|     Dept|count|
+---------+-----+
|    Sales|    2|
|Marketing|    2|
+---------+-----+



explain function is used to get the plan that is used for the query

In [9]:
result.explain(False)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Dept#86], functions=[count(1)])
   +- Exchange hashpartitioning(Dept#86, 200), ENSURE_REQUIREMENTS, [plan_id=103]
      +- HashAggregate(keys=[Dept#86], functions=[partial_count(1)])
         +- Project [Dept#86]
            +- Filter (isnotnull(Status#85) AND (Status#85 = A))
               +- Scan ExistingRDD[Name#80,Emp_id#81L,Joining_date#82,DOB#83,Salary#84L,Status#85,Dept#86]




    To get deatiled plan

In [11]:
result.explain(True)

== Parsed Logical Plan ==
Aggregate [Dept#86], [Dept#86, count(1) AS count#131L]
+- Filter (Status#85 = A)
   +- LogicalRDD [Name#80, Emp_id#81L, Joining_date#82, DOB#83, Salary#84L, Status#85, Dept#86], false

== Analyzed Logical Plan ==
Dept: string, count: bigint
Aggregate [Dept#86], [Dept#86, count(1) AS count#131L]
+- Filter (Status#85 = A)
   +- LogicalRDD [Name#80, Emp_id#81L, Joining_date#82, DOB#83, Salary#84L, Status#85, Dept#86], false

== Optimized Logical Plan ==
Aggregate [Dept#86], [Dept#86, count(1) AS count#131L]
+- Project [Dept#86]
   +- Filter (isnotnull(Status#85) AND (Status#85 = A))
      +- LogicalRDD [Name#80, Emp_id#81L, Joining_date#82, DOB#83, Salary#84L, Status#85, Dept#86], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Dept#86], functions=[count(1)], output=[Dept#86, count#131L])
   +- Exchange hashpartitioning(Dept#86, 200), ENSURE_REQUIREMENTS, [plan_id=103]
      +- HashAggregate(keys=[Dept#86], functions=[partial

    it will provide us the steps, how spark engine is approaching for agiven query.

    Always read the plan from bottom to top.

    Plan is followed from bottom to top

**END OF CODE**